# Libraries load 

In [55]:
import json
from scipy.optimize import newton_krylov
from pyomo.environ import SolverFactory, SolverManagerFactory
from pyomo.environ import*
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pulp import LpVariable, LpMinimize, LpProblem, lpSum, value, GUROBI, GLPK_CMD
import envs as en
import pyomo.contrib.solver.ipopt
import os
from pyomo.environ import value

from io import BytesIO
from typing import List, Optional, Dict, Any
from pydantic import BaseModel

import requests

# Input file load


In [56]:
""" input_file = "DataSets/V3/sampledata4.xlsx"

# Spanish prosumer comunity example
prosumer_id_mapping = {
        0: "abc1",
        1: "mnl2",
        2: "xyz3",
        3: "def4",
        4: "ghi5"
    } """

' input_file = "DataSets/V3/sampledata4.xlsx"\n\n# Spanish prosumer comunity example\nprosumer_id_mapping = {\n        0: "abc1",\n        1: "mnl2",\n        2: "xyz3",\n        3: "def4",\n        4: "ghi5"\n    } '

# Code Methods


## Active atributes of a prosumer

Contains a string which represents the prosumer identification and is associated with a key (integer number), and a boolean for each column of data that is activated (true / value =! 0) or deactivated (false/value==0).


In [57]:
class ActiveAttribute(BaseModel):
    prosumerId: str
    profileLoad: bool
    stateOfCharge: bool
    photovoltaicEnergyLoad: bool

## Load excel file

    Load all sheets from an Excel file into a dictionary of DataFrames.
    
    Args:
        file_path (str): Path to the Excel file.
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary mapping sheet names to their DataFrames.
        
    Raises:
        FileNotFoundError: If the file does not exist.
        ValueError: If the file cannot be read or required sheets are missing.

In [58]:
def load_excel_file(file_path: str) -> Dict[str, pd.DataFrame]:

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Excel file not found at: {file_path}")
    
    try:
        # Load all sheets
        xl = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')
        required_sheets = ['PL', 'PPV_capacity']
        for sheet in required_sheets:
            if sheet not in xl:
                raise ValueError(f"Required sheet '{sheet}' not found in the Excel file.")
        return xl
    except Exception as e:
        raise ValueError(f"Failed to read Excel file: {str(e)}")

## Print DataFrame

    Prints each DataFram within a dictionary, identifying by the name of the key.
    

In [59]:
from typing import Dict
import pandas as pd

def print_dataframes_dict(dataframes: Dict[str, pd.DataFrame]):
    for name, df in dataframes.items():
        print(f"\n=== DataFrame: {name} ===")
        print(df.head())  # ou apenas print(df) se quiser tudo


## Check zero columns

    Check which columns in the specified DataFrame have all values equal to zero.
    
    Args:
        df (pd.DataFrame): DataFrame of the sheet to check.
        sheet_name (str): Name of the sheet ('PL' or 'PPV_capacity').
        
    Returns:
        List[str]: List of column names with all values equal to zero.
        
    Raises:
        ValueError: If the sheet name is invalid or columns are not found.

In [60]:
def check_zero_columns(df: pd.DataFrame, sheet_name: str) -> List[str]:

    valid_columns = {
        'PL': [col for col in df.columns if col.startswith('PL')],
        'PPV_capacity': [col for col in df.columns if col.startswith('PV')]
    }
    
    if sheet_name not in valid_columns:
        raise ValueError(f"Invalid sheet name: {sheet_name}. Must be 'PL' or 'PPV_capacity'.")
    
    zero_columns = []
    for col in valid_columns[sheet_name]:
        if col in df.columns and (df[col] == 0).all():
            zero_columns.append(col)
    
    return zero_columns

## Set Column to zero 

    Set all values in the specified column of the DataFrame to zero.
    
    Args:
        df (pd.DataFrame): DataFrame of the sheet to modify.
        sheet_name (str): Name of the sheet ('PL' or 'PPV_capacity').
        column_name (str): Name of the column to set to zero (e.g., 'PL1', 'PV2').
        
    Returns:
        pd.DataFrame: Modified DataFrame with the specified column set to zero.
        
    Raises:
        ValueError: If the sheet name or column name is invalid.

In [61]:
def set_column_to_zero(df: pd.DataFrame, sheet_name: str, column_name: str) -> pd.DataFrame:

    valid_columns = {
        'PL': [col for col in df.columns if col.startswith('PL')],
        'PPV_capacity': [col for col in df.columns if col.startswith('PV')]
    }
    
    if sheet_name not in valid_columns:
        raise ValueError(f"Invalid sheet name: {sheet_name}. Must be 'PL' or 'PPV_capacity'.")
    
    if column_name not in valid_columns[sheet_name]:
        raise ValueError(f"Invalid column name: {column_name}. Must be one of {valid_columns[sheet_name]}.")
    
    if column_name not in df.columns:
        raise ValueError(f"Column {column_name} not found in sheet {sheet_name}.")
    
    df[column_name] = 0
    return df

## Save excel file 

    Save the modified DataFrames back to an Excel file.
    
    Args:
        sheets (Dict[str, pd.DataFrame]): Dictionary of sheet names and their DataFrames.
        output_path (str): Path to save the modified Excel file.
        
    Raises:
        ValueError: If saving the file fails.


In [62]:
def save_excel_file(sheets: Dict[str, pd.DataFrame], output_path: str) -> None:

    try:
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            for sheet_name, df in sheets.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Excel file saved successfully to: {output_path}")
    except Exception as e:
        raise ValueError(f"Failed to save Excel file: {str(e)}")

## Generate active atributes
  
    Generate a list of ActiveAttribute objects based on zero columns in PL and PPV_capacity sheets,
    using a predefined mapping of integer indices to prosumer IDs.
    
    Args:
        sheets (Dict[str, pd.DataFrame]): Dictionary of sheet names and their DataFrames.
        prosumer_id_mapping (Dict[int, str]): Mapping of integer indices (e.g., 0, 1) to prosumer IDs (e.g., 'abc1', 'mnl2').
        
    Returns:
        List[ActiveAttribute]: List of ActiveAttribute objects with updated profileLoad and photovoltaicEnergyLoad.
        
    Raises:
        ValueError: If required sheets are missing or invalid.
   

In [63]:


def generate_active_attributes(sheets: Dict[int, pd.DataFrame], prosumer_id_mapping: Dict[int, str]) -> List[ActiveAttribute]:
    # Check zero columns
    pl_zero = check_zero_columns(sheets['PL'], 'PL')
    ppv_zero = check_zero_columns(sheets['PPV_capacity'], 'PPV_capacity')
    
    active_attributes = []
    for index, prosumer_id in prosumer_id_mapping.items():
        # Convert index to column number (index 0 -> PL1, PV1)
        col_number = index + 1
        pl_column = f"PL{col_number}"
        pv_column = f"PV{col_number}"
        
        # Set profileLoad to False if PL column is all zeros
        profile_load = pl_column not in pl_zero if pl_column in sheets['PL'].columns else False
        
        # Set photovoltaicEnergyLoad to False if PV column is all zeros
        pv_load = pv_column not in ppv_zero if pv_column in sheets['PPV_capacity'].columns else False
        
        # Assume stateOfCharge is True by default (can be modified based on ESS-Param if needed)
        state_of_charge = True
        
        active_attribute = ActiveAttribute(
            prosumerId=prosumer_id,
            profileLoad=profile_load,
            stateOfCharge=state_of_charge,
            photovoltaicEnergyLoad=pv_load
        )
        active_attributes.append(active_attribute)
    
    return active_attributes

## Main

In [ ]:
def main():
    # File paths
    input_file = "sampledata4.xlsx"
    output_file = "modified_sampledata4.xlsx"
    
    # Sample prosumer ID mapping with integer keys
    prosumer_id_mapping = {
        0: "abc1",
        1: "mnl2",
        2: "xyz3",
        3: "def4",
        4: "ghi5",
    }
    
    # Load the Excel file
    try:
        sheets = load_excel_file(input_file)
        #print_dataframes_dict(sheets)  # Print the loaded DataFrames
    except Exception as e:
        print(f"Error loading file: {str(e)}")
        return
    
    # Check columns with all zero values in PL and PPV_capacity
    try:
        pl_zero = check_zero_columns(sheets['PL'], 'PL')
        ppv_zero = check_zero_columns(sheets['PPV_capacity'], 'PPV_capacity')
        
        print("Columns with all zero values in PL sheet:", pl_zero)
        print("Columns with all zero values in PPV_capacity sheet:", ppv_zero)
    except ValueError as e:
        print(f"Error checking zero columns: {str(e)}")
        return
    
    # Generate active_attributes list
    try:
        active_attributes = generate_active_attributes(sheets, prosumer_id_mapping)
        print("Active Attributes:")
        for attr in active_attributes:
            print(f"Prosumer {attr.prosumerId}: profileLoad={attr.profileLoad}, "
                  f"stateOfCharge={attr.stateOfCharge}, photovoltaicEnergyLoad={attr.photovoltaicEnergyLoad}")
    except ValueError as e:
        print(f"Error generating active attributes: {str(e)}")
        return
    
    # Example: Set PL2 and PV2 to zero
    try:
        sheets['PL'] = set_column_to_zero(sheets['PL'], 'PL', 'PL1')
        sheets['PPV_capacity'] = set_column_to_zero(sheets['PPV_capacity'], 'PPV_capacity', 'PV2')
        print("Set PL2 to zero successfully.")
        active_attributes = generate_active_attributes(sheets, prosumer_id_mapping)
        print("\nNew Active Attributes:")
        for attr in active_attributes:
            print(f"Prosumer {attr.prosumerId}: profileLoad={attr.profileLoad}, "
                  f"stateOfCharge={attr.stateOfCharge}, photovoltaicEnergyLoad={attr.photovoltaicEnergyLoad}")
    except ValueError as e:
        print(f"Error setting column to zero: {str(e)}")
        return
    
    # Save the modified Excel file
    try:
        save_excel_file(sheets, output_file)
    except ValueError as e:
        print(f"Error saving file: {str(e)}")
        return

if __name__ == "__main__":
    main()

Columns with all zero values in PL sheet: []
Columns with all zero values in PPV_capacity sheet: ['PV3', 'PV4']
Active Attributes:
Prosumer abc1: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=True
Prosumer mnl2: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=True
Prosumer xyz3: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=False
Prosumer def4: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=False
Prosumer ghi5: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=True
Set PL2 to zero successfully.

Active Attributes:
Prosumer abc1: profileLoad=False, stateOfCharge=True, photovoltaicEnergyLoad=True
Prosumer mnl2: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=False
Prosumer xyz3: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=False
Prosumer def4: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=False
Prosumer ghi5: profileLoad=True, stateOfCharge=True, photovoltaicEnergyLoad=True
Exc